In [13]:
#Imports
from deap import creator, tools, base
import numpy as np
import timeit
from stage import fullPrint, getStage
from datetime import datetime, date, timedelta
from sklearn.base import BaseEstimator
import random
import pandas as pd
import yahoo_fin.stock_info as yf
from sklearn.model_selection import train_test_split
import pickle
# here
# from multiprocessing import Pool

In [14]:
start = timeit.default_timer()
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

/Users/alanhou/opt/anaconda3/envs/titanic/lib/python3.9/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/Users/alanhou/opt/anaconda3/envs/titanic/lib/python3.9/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


In [15]:
nasdaqList = pd.read_pickle("stockData/tickerList.pkl")

train, test = train_test_split(nasdaqList, test_size=0.3, shuffle=True)
# deleteAfterTest
train, test = train_test_split(nasdaqList[0:3000], test_size=0.3, shuffle=True)
#end
trainSet1, trainSet2, trainSet3  = np.array_split(train,3)
trainSets = [trainSet1, trainSet2, trainSet3]
resultFile = open("resultML.txt","a")
resultFile.write("train "+str(train)+"\n")
resultFile.write("test "+str(test)+"\n")
resultFile.close()

now = datetime.now()
transactionTemplate = yf.get_data('AAPL', start_date="1995-01-06",end_date= now, index_as_date = True).drop(['open','high','low','close','adjclose','volume','ticker'],axis=1)
transactionTemplate['Dates'] = pd.to_datetime(transactionTemplate.index)
transactionTemplate = transactionTemplate[transactionTemplate['Dates'].dt.weekday == 4]
transactionTemplate = transactionTemplate.drop('Dates', axis = 1)
transactionTemplate.to_pickle("transactionTemplate.pkl")


In [16]:
removeList = []
def calculateGroupReturn(list):
    monthlyReturn = pd.read_pickle("transactionTemplate.pkl")
    monthlyReturn = monthlyReturn.resample('M').mean()
    monthlyReturn.index = monthlyReturn.index.strftime("%Y:%m")
    currentColumn = -1
    for symbol in list:
        if symbol in monthlyReturn.columns:
            continue
        try:
            df = pd.read_pickle("stockData/nyseNasdaq/"+symbol+".pkl")
        except:
            removeList.append(symbol)
            print(symbol)
            continue
        if not (df['close'] != 0).all() or df.empty:
            if df.empty:
                print(symbol)
                removeList.append(symbol)
            continue
        monthlyReturn[symbol] = 1
        currentColumn += 1
        monthCheck = df.index[0].strftime("%Y:%m")
        lastPrice = df.iat[0,0]
        initialPrice = df.iat[0,0]
        for index, element in df.iterrows():
            if (index.strftime("%Y:%m") != monthCheck):
                if (lastPrice / initialPrice >= 1.01 and lastPrice / initialPrice < 2):
                    rowNumber = monthlyReturn.index.get_loc(index.strftime("%Y:%m"))
                    monthlyReturn.iat[rowNumber, currentColumn] = (lastPrice / initialPrice)
                initialPrice = element[0]
                lastPrice = element[0]
                monthCheck = index.strftime("%Y:%m")
            lastPrice = element[0]
    return monthlyReturn
            
import warnings
warnings.filterwarnings("ignore", category=pd.errors.PerformanceWarning) 
i = 0
listOfDf = calculateGroupReturn(train)
for element in removeList:
    nasdaqList.remove(element)
with open('stockData/tickerList.pkl', 'wb') as f:
    pickle.dump(nasdaqList, f)



In [17]:
indexNum = 0
sum = [1,1,1,1,1,1]
for i in range(6):
    if i < 3:
        l = trainSets[i]
    else:
        l = np.concatenate((trainSets[(i+1)%3],trainSets[(i+2)%3]))
    index = 0
    while index != len(l):
        if l[index] not in listOfDf.columns:
            l = np.delete(l, index)
        else:
            index += 1
    for index, element in listOfDf[l].iterrows():
        #print(element.to_list())
        listOfStockRet = element.to_list()
        while 1.0 in listOfStockRet:
            listOfStockRet.remove(1.0)
        if len(listOfStockRet) != 0:
            sum[i] = sum[i] * np.mean(listOfStockRet)     
for i in range(1,6):
    sum[i] = sum[i] / sum[0]
sum[0] = 1.0
print(sum)

[1.0, 0.6742404008647392, 0.7626481017441731, 0.7390449828878252, 0.8732954474293788, 0.8239409711504343]


In [18]:
def generate_random_num_attr():
  original = np.array([0,0,0,0,0,0,0,0,0,1,0.1,1,0.995,0.9,0.985])
  noises = []
  sum_list = []

  noises.append(random.uniform(0,1))
  noises.append(random.uniform(0.1,2.5))
  noises.append(random.uniform(-0.1,0.4))
  noises.append(random.uniform(0.9,1.1))
  noises.append(random.uniform(1,1.4))
  noises.append(random.uniform(1, 1.3))
  noises.append(random.uniform(0.8, 1))
  noises.append(random.uniform(0.8, 1.1))
  noises.append(random.uniform(1.2, 1.5))
  noises.append(np.random.normal(0, 0.001))
  noises.append(np.random.normal(0, 0.1))

  noises.append(np.random.normal(0, 0.05))
  noises.append(np.random.normal(0, 0.004))
  noises.append(np.random.normal(0, 0.05))
  noises.append(np.random.normal(0, 0.004))

  while original[11] + noises[11] < original[13] + noises[13]:
    noises[11] = np.random.normal(0, 0.05)
    noises[13] = np.random.normal(0, 0.05)
  while original[12] + noises[12] < original[14] + noises[14]:
    noises[12] = np.random.normal(0, 0.004)
    noises[14] = np.random.normal(0, 0.004)
  
  for (item1, item2) in zip(original, noises):
      sum_list.append(item1+item2)
  
  return sum_list
noises = []
noises.append(np.random.normal(0, 0.025))
noises.append(np.random.normal(0, 0.075))
noises.append(np.random.normal(0, 0.015))
noises.append(np.random.normal(0, 0.005))
noises.append(np.random.normal(0, 0.0125))
noises.append(np.random.normal(0, 0.015))
noises.append(np.random.normal(0, 0.005))
noises.append(np.random.normal(0, 0.01))
noises.append(np.random.normal(0, 0.0125))
noises.append(np.random.normal(0, 0.0005))
noises.append(np.random.normal(0, 0.0025))

noises.append(np.random.normal(0, 0.0025))
noises.append(np.random.normal(0, 0.0002))
noises.append(np.random.normal(0, 0.0025))
noises.append(np.random.normal(0, 0.0002))
def mutate(individual):
  size = len(individual)
  for i in range(0,size):
    if random.random() > 0.9:
      individual[i] = individual[i] + noises[i]
  return individual,

In [19]:
toolbox = base.Toolbox()
# Attribute generator 
toolbox.register("attr_bool", generate_random_num_attr)
# Structure initializers
toolbox.register("individual", tools.initRepeat, creator.Individual, 
    toolbox.attr_bool, 1)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In [20]:
SECTORS = 6
def evalReturn(individual):
    print(individual[0])
    now = datetime.now()
    now = now.strftime("%Y-%m-%d-%H:%M")
    transaction = pd.read_pickle("transactionTemplate.pkl")
    transaction['holding'] = np.empty((len(transaction), 0)).tolist()
    SectorDict = {
    "Energy":"XLE","Technology":"XLK",
    "Communication Services":"XLC",
    "Consumer Staples":"XLP",
    "Health Care":"XLV",
    "Consumer Cyclical":"XLY",
    "Industrial":"XLI",
    "Utilities":"XLU",
    "Materials":"XLB",
    "Financial":"XLF",
    "Real Estate":"XLRE"}
    sectorDfList = []
    for sector in SectorDict.values():
        sectorDfList.append(pd.read_pickle("stockData/sectorCharts/"+sector+".pkl"))
        goodSectorDf = sectorDfList[0].drop(['open','high','low','adjclose','close','ticker','volume','percent','MA30','MA30Slope','RS'],axis=1)
        goodSectorDf['Sectors'] = [[] for _ in range(len(goodSectorDf))]
        goodSectorDf.index = sectorDfList[0].index
        index = 0
        for date in sectorDfList[0].index:
            listOfSector = []
            for df in sectorDfList:
                try:
                    # print(df.at[date,'MA30Slope'],df.at[date,'RS'])
                    # print(individual[0][9],individual[0][10])
                    if  df.at[date,'MA30Slope'] > individual[0][9] or df.at[date,'RS'] > individual[0][10]:
                        if df.at[date,'close'] > df.at[date,'MA30']:
                            # print(df.at[date,'ticker'])
                            listOfSector.append(df.at[date,'ticker'])
                except: 
                    pass
                # print(listOfSector)
            goodSectorDf.iat[index,0] = set(listOfSector)
            index = index + 1
    stockStageEstimator = StockStageEstimator(individual[0], goodSectorDf, trainSets)
    scores = stockStageEstimator.score()
    result = stockStageEstimator.result()
    print(scores)
    print(np.average(scores))
    print(result)
    print(stockStageEstimator.getReturns())
    if scores == -1:
        return 0,
    if np.average(scores) >  0.75:
        return 0,
    paramFile = open("params.txt","a")
    paramFile.write(str(individual[0])+"\n")
    paramFile.write(str(result)+"\n")
    paramFile.close()
    resultFile = open("resultML.txt","a")
    resultFile.write(str(individual[0])+"\n")
    resultFile.write(str(np.average(scores))+"\n")
    resultFile.write(str(stockStageEstimator.getReturns())+"\n")
    resultFile.write(str(result)+"\n")
    resultFile.close()
    return result,

In [21]:
toolbox.register("evaluate", evalReturn)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", mutate)
toolbox.register("select", tools.selTournament, tournsize=3)

In [22]:
HOLDING = 0
TOTAL = 1
class StockStageEstimator(BaseEstimator):
    def __init__(self, paramList = [0.20, 1.39, 0.35, 0.93, 1.09, 1.22, 0.98, 0.98, 1.38, 0.99, -0.01, 0.97, 0.99, 0.96, 0.98], goodSectorDf = pd.DataFrame, sets = [[]]):
        self.paramList = paramList
        self.goodSectorDf = goodSectorDf
        self.returns = [0,0,0,0,0,0]
        self.sets = sets
        self.scores = [0,0,0]
    
    def evalFit(self, tickers, goodSectorDf):
        transactionFit = pd.read_pickle("transactionTemplate.pkl")
        transactionFit['holding'] = np.empty((len(transactionFit), 0)).tolist()
        #print("Reach #1")
        for symbol in tickers:
            df = getStage(symbol,self.paramList,goodSectorDf)
            buyDf = pd.read_pickle('stockData/nysenasdaq/'+symbol+'Buy.pkl')
            inStage = False
            buyTwice = False
            if df.empty:
                continue
            for index, element in df.iterrows():
                open = 0
                monday = index + timedelta(3)
                i = transactionFit.index.get_loc(index)
                if element.Stage == "Stage 2" or element.Stage == "Buy":
                    if monday in buyDf.index:
                        open = buyDf.at[monday,'open']
                    else:
                        open = element.close
                    transactionFit.iat[i,HOLDING].append((symbol,open,0))
                    if buyTwice:
                        transactionFit.iat[i, HOLDING].append((symbol,open,0))
                    if element.Stage == "Buy" and inStage:
                        transactionFit.iat[i, HOLDING].append((symbol,open,0))
                        buyTwice = True
                    inStage = True
                    continue
                if "Sell" == element.Stage[0:4]:
                    if monday in buyDf.index:
                        open = buyDf.at[monday,'open']
                    else:
                        open = element.close
                    transactionFit.iat[i, HOLDING].append((symbol,open,-1))
                    if buyTwice:
                        transactionFit.iat[i, HOLDING].append((symbol,open,-1))
                    inStage = False
                    buyTwice = False
        #print("Reach #2")
        total = 100
        first = True
        transactionFitCopy = transactionFit
        transactionFitCopy['total'] = 0
        transactionFitCopy.to_csv("onlyShare.csv")
        #print("Reach #3")
        for index, element in transactionFitCopy.iterrows():
            indexNum = transactionFitCopy.index.get_loc(index)
            if first:
                first = False
                transactionFitCopy.iat[indexNum,1] = 100
                # print(i,transactionFitCopy.iat[i,1],"#1")
                continue
            if len(element.holding) == 0:
                transactionFitCopy.iat[indexNum,1] = transactionFitCopy.iat[indexNum-1,1]
                # print(i,transactionFitCopy.iat[i,1],"#2")
                continue
            else:
                prevData = transactionFitCopy.iat[indexNum-1, 0]
                if len(prevData):
                    total = 0
                    removeList = []
                    close = 0
                    for i in range(len(prevData)):
                        for j in range(len(element.holding)):
                            if element.holding[j][0] == prevData[i][0]:
                                close = element.holding[j][1]
                                if element.holding[j][2] == -1:
                                    removeList.append(element.holding[j])
                                break
                        total += close*prevData[i][2]
                    transactionFitCopy.iat[indexNum,1] = total
                    # print(i,transactionFitCopy.iat[indexNum,1],"#3")
                    for delete in removeList:
                        if delete in element.holding:
                            element.holding.remove(delete)
                else:
                    transactionFitCopy.iat[indexNum,1] = transactionFitCopy.iat[indexNum-1,1]
                    # print(i,transactionFitCopy.iat[i,1],"#4")
            if len(element.holding):
                allocation = total/len(element.holding)
            for i in range(len(element.holding)):
                element.holding[i] = (element.holding[i][0],element.holding[i][1],allocation/element.holding[i][1])
        stockHolding  = 0
        transactionFitCopy.to_csv("estimatorTest.csv")
        for i in transactionFitCopy.iterrows():
            stockHolding += len(i[1]['holding'])
        if stockHolding/(transactionFitCopy.shape[0])<(len(tickers)/1000) or transactionFitCopy.iat[-1,TOTAL]<=100:
            print(stockHolding)
            print(transactionFitCopy.iloc[-1]['total'])
            return -1
        return transactionFitCopy.iloc[-1]['total']

    def fit(self):
        for i in range(3):
            setsCombined = np.concatenate((self.sets[(i+1)%3],self.sets[(i+2)%3]))
            self.returns[i*2] = self.evalFit(setsCombined, self.goodSectorDf)
            if self.returns[i*2] == -1:
                return -1
            self.returns[i*2+1] = self.evalFit(self.sets[i%4], self.goodSectorDf)
            if self.returns[i*2+1] == -1:
                return -1
            
        return 0
    
    def score(self):
        if self.fit() == -1:
            return -1
        for i in range(3):
            self.scores[i] = np.absolute(1-((self.returns[i*2])/sum[i]/(self.returns[i*2+1])/sum[3+i]))
        return self.scores
    
    def result(self):
        return np.average(self.returns)
    
    def getReturns(self):
        return self.returns

In [23]:
def main():
    pop = toolbox.population(n=128)
    # Evaluate the entire population
    # here
    # pool = Pool()
    # tempResult = pool.map(toolbox.evaluate, pop)
    tempResult = map(toolbox.evaluate, pop)
    print(tempResult)
    fitnesses = list(tempResult)
    # here
    # pool.close()
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit
        if ind.fitness.values[0]<=100:
            del ind.fitness.values
    badInd = [ind for ind in pop if not ind.fitness.valid]
    while len(badInd)!=0:
        for ind in badInd:
            index = pop.index(ind)
            temp = toolbox.individual()
            pop[index] = temp
            del pop[index].fitness.values
        badInd = [ind for ind in pop if not ind.fitness.valid]
        fitnesses = map(toolbox.evaluate, badInd)
        for ind, fit in zip(badInd, fitnesses):
            ind.fitness.values = fit
            if ind.fitness.values[0]<=100:
                del ind.fitness.values
        badInd = [ind for ind in pop if not ind.fitness.valid]
    
        
    # Begin the evolution
    for g in range(7):
        np.random.shuffle(train)
        trainSet1, trainSet2, trainSet3  = np.array_split(train,3)
        trainSets = [trainSet1, trainSet2, trainSet3]
        print("-- Generation %i --" % g)
        resultFile = open("resultML.txt","a")
        resultFile.write("-- Generation %i --" % g+"\n")
        resultFile.close()
        
        # Select the next generation individuals
        offspring = toolbox.select(pop, len(pop))
        # Clone the selected individuals
        offspring = list(map(toolbox.clone, offspring))

        # Apply crossover and mutation on the offspring
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < 0.5:
                toolbox.mate(child1[0], child2[0])
                del child1.fitness.values
                del child2.fitness.values

        for mutant in offspring:
            if random.random() < 0.2:
                toolbox.mutate(mutant[0])
                del mutant.fitness.values        
    
        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        # here
        # pool = Pool()
        # fitnesses = pool.map(toolbox.evaluate, invalid_ind)
        # pool.close()
        fitnesses = map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit
        
        # Replace population
        pop[:] = offspring
    
        # Gather all the fitnesses in one list and print the stats
        fits = [ind.fitness.values[0] for ind in pop]
        
        length = len(pop)
        mean = sum(fits) / length
        sum2 = sum(x*x for x in fits)
        std = abs(sum2 / length - mean**2)**0.5
        
        print("  Min %s" % min(fits))
        print("  Max %s" % max(fits))
        print("  Avg %s" % mean)
        print("  Std %s" % std)
        
        
    print("-- End of (successful) evolution --")
    
    best_ind = tools.selBest(pop, 1)[0]
    print("Best individual is %s, %s" % (best_ind, best_ind.fitness.values))


In [24]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
main()

[0.7727781685241863, 1.2699597625360213, -0.04890849112891668, 0.9849649291227153, 1.1310815499127505, 1.2222091860881203, 0.8515778202370214, 0.8110054157290518, 1.4437394862878996, 1.0005059100928742, 0.2242341435168176, 0.9906153295859251, 0.991998728236804, 0.8403479065529125, 0.9870723643600392]
[1.7244884072196887, 3.6392920442387346, 0.46658837900726535]
1.9434562768218961
665.5549636374888
[694.3280025018041, 344.83301827431734, 850.0325587596795, 311.177279934033, 450.1032150302551, 1342.8557073248435]
[0.362053446446737, 0.25591315621984717, 0.2263094238409177, 0.9479295558218191, 1.1005375787401384, 1.2132452109273033, 0.8446740210097636, 0.8458780733930067, 1.4447561994617844, 1.0004751019235598, 0.23294941292458737, 1.0763851584487218, 0.9902770492443024, 0.8805714400186964, 0.9852537365146985]
7052
90.99529972422962
           holding     total
1995-01-06      []  100.0000
1995-01-13      []  100.0000
1995-01-20      []  100.0000
1995-01-27      []  100.0000
1995-02-03   